In [ ]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.preprocessing.sequence import TimeseriesGenerator
import matplotlib.pyplot as plt

In [ ]:
df_forecast = pd.read_csv("dataset5_important_features.csv", parse_dates=['datetime'])
df_forecast = df_forecast.sort_values('datetime')

Resampling hourly just to be sure

In [ ]:
df_forecast = df_forecast.set_index('datetime').resample('1h').mean().dropna()
df_forecast.reset_index(inplace=True)

In [ ]:
target_col = 'svr1'
generator = TimeseriesGenerator(
    df_forecast.drop(columns=[target_col, 'datetime']).values,
    df_forecast[target_col].values,
    length=24,
    batch_size=32
)

Simple LSTM

In [ ]:
model = Sequential([
    LSTM(50, activation='relu', input_shape=(24, df_forecast.drop(columns=[target_col, 'datetime']).shape[1])),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')

In [ ]:
history = model.fit(generator, epochs=50, verbose=2)
plt.plot(history.history['loss'])
plt.title("LSTM Training Loss")
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.savefig("lstm_training_loss.png")
plt.show()